In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from bs4 import BeautifulSoup
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import numpy as np
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import xlsxwriter
from openpyxl import Workbook, load_workbook
import os
import shutil
import logging
import traceback
import re
import csv
from azure.storage.blob import BlobServiceClient
# 

In [80]:
# File paths


# date_today = datetime.date.today().strftime("%Y-%m-%d")
# output_gyg = r'output/GYG'
# archive_folder = fr'{output_gyg}/Archive'
# file_path_done =fr'output/GYG/{date_today}-DONE-GYG.csv'  
# file_path_output = fr"output/GYG - {date_today}.xlsx"
# link_file = fr'resource/GYG_links.csv'
# avg_file = fr'resource/avg-gyg.csv'
# re_run_path = fr'output/GYG/{date_today}-ReRun-GYG.csv'
# folder_path_with_txt_to_count_avg = 'Avg/GYG'

date_today = datetime.date.today().strftime("%Y-%m-%d")
# date_today = '2024-02-23'
output_gyg = r'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Get Your Guide/future_price'
archive_folder = fr'{output_gyg}/Archive'
file_path_done =fr'{output_gyg}/{date_today}-DONE-GYG.csv'  
file_path_output = fr"{output_gyg}/GYG - {date_today}.xlsx"
link_file = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/LinksFuturePrice_GYG.csv'
logs_path = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/GYG/future_price'

# Set the path of the local file
local_file_path = f"{output_gyg}/{date_today}_future_price.xlsx"

# Set the name of your Azure Storage account and the corresponding access key
storage_account_name = "storagemyotas"
storage_account_key = "vyHHUXSN761ELqivtl/U3F61lUY27jGrLIKOyAplmE0krUzwaJuFVomDXsIc51ZkFWMjtxZ8wJiN+AStbsJHjA=="

# Set the name of the container and the desired blob name
container_name_raw = "raw/future_price/gyg"
container_name_refined = "refined/future_price/gyg"

blob_name = fr'{date_today}_future_price.xlsx'
file_path_logs_processed = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/files_processed/{blob_name.split(".")[0]}'


In [81]:
# create logger object
logger_err = logging.getLogger('Error_logger')
logger_err.setLevel(logging.DEBUG)
logger_info = logging.getLogger('Info_logger')
logger_info.setLevel(logging.DEBUG)
logger_done = logging.getLogger('Done_logger')
logger_done.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create file handler for error logs and set level to debug
fh_error = logging.FileHandler(fr'{logs_path}/error_logs.log')
fh_error.setLevel(logging.DEBUG)

# create file handler for info logs and set level to info
fh_info = logging.FileHandler(fr'{logs_path}/info_logs.log')
fh_info.setLevel(logging.INFO)

# create file handler for info logs and set level to info
fh_done = logging.FileHandler(fr'{logs_path}/done_logs.log')
fh_done.setLevel(logging.INFO)
# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to handlers
ch.setFormatter(formatter)
fh_error.setFormatter(formatter)
fh_info.setFormatter(formatter)
fh_done.setFormatter(formatter)

# add handlers to logger
logger_err.addHandler(ch)
logger_err.addHandler(fh_error)
logger_info.addHandler(ch)
logger_info.addHandler(fh_info)
logger_done.addHandler(ch)
logger_done.addHandler(fh_done)

In [82]:
def handle_error_and_rerun(error):
#     recipient_error = 'wojbal3@gmail.com'
    tb = traceback.format_exc()
    logger_err.error('An error occurred: {} on {}'.format(str(error), tb))
#     subject = f'Error occurred - {time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}'
#     message = f'<html><body><p>Error occurred: {str(error)} on {tb}</p></body></html>'
#     send_email(subject, message, recipient_error)

In [83]:
def create_log_done(log_type):
    global file_path_logs_processed
    if log_type == 'Raw':
        with open(f'{file_path_logs_processed}-raw.txt', 'w') as file:
            file.write('Done')
    elif log_type == 'Refined':
        with open(f'{file_path_logs_processed}-refined.txt', 'w') as file:
            file.write('Done')

In [84]:
def initilize_driver() -> WebDriver:
    try:
        logger_info.info("Initializing the Chrome driver and logging into the website")

        # Setting up Chrome options
        options = webdriver.ChromeOptions()
        # options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--blink-settings=imagesEnabled=true')
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        

        # Initialize the Chrome driver
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.maximize_window()
        
        return driver

    except Exception as e:
        logger_err.error(f"An error occurred during login: {e}")
        raise
    
def quit_driver(driver: WebDriver) -> None:
    driver.quit()
    

In [85]:
def get_first_days_of_current_and_next_six_months() -> list:
    try:
        logger_info.info("Starting to get first days of current and next six months")

        # Current date
        current_date = datetime.now()

        # List to store first days of the current and next six months
        first_days = []

        # Add the first day of the current month
        first_day_current_month = datetime(current_date.year, current_date.month, 1)
        first_days.append(first_day_current_month.strftime("%Y-%m-%d"))

        # Add the first days of the next six months
        for month in range(1, 7):
            # Calculate the first day of each future month
            year = current_date.year
            month_num = current_date.month + month

            # Adjust for year change
            if month_num > 12:
                month_num -= 12
                year += 1

            first_day = datetime(year, month_num, 1)
            first_days.append(first_day.strftime("%Y-%m-%d"))

        logger_done.info("Successfully retrieved first days of current and next six months")
        return first_days

    except Exception as e:
        logger_err.error(f"Error in get_first_days_of_current_and_next_six_months: {e}")
        raise
    

In [ ]:
def change_currency(currency_switcher_button, currency):
    currency_text = currency_switcher_button.text.strip()
    if currency in currency_text:
        print('Current currency is ',currency)
        return
    else: 
                # hover over the currency switcher button to show the menu
        actions = ActionChains(driver)
        actions.move_to_element(currency_switcher_button).perform()
        currency_switcher_button .click()
        # wait for the EUR currency option to be clickable
        eur_currency_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//li[@class='currency-modal-picker__item-parent item__currency-modal item__currency-modal--EUR']")))
        # click on the EUR currency option to change the currency
        eur_currency_option.click()
def save_and_erase_dataframe(df: pd.DataFrame, url_unique_identifier) -> pd.DataFrame:
    ## SAVE
    file_path = fr'{output_gyg}/{date_today}-{url_unique_identifier}-GYG.csv' 
    df['city'] = url_unique_identifier.split('-')[0]
    df.to_csv(file_path, header=not os.path.exists(file_path), index=False, mode='a')
    logger_done.info(f'Sucessfully upserted {len(df)} rows to {file_path}')
    return pd.DataFrame()
def extract_options(option_detials) -> list:
    list_of_items = []
    #iterate through all options
    time.sleep(1)
    option_date = driver.current_url.split('date_from=')[-1].split('&')[0]
    
    for option in option_detials:
        option_title = option.find_element(By.CLASS_NAME, 'activity-option__title').text
        try:
            option_time_range = option.find_element(By.CLASS_NAME, 'activity-option__start-time-range').text
        except:
            option_time_range = 'Not listed'
        try:
            option_price_total = option.find_element(By.CLASS_NAME, 'activity-option-price-wrapper__price').text
            option_price_per_person = float(option.find_element(By.CLASS_NAME, 'activity-option-price-wrapper__price').text.replace('€', '').replace(',', '').strip()) / float(adults_amount)
        except:
            try:
                option_price_total = option.find_element(By.CLASS_NAME, 'activity-option__cart-message-text--future-availability').text
            except:
                option_price_total = option.find_element(By.CLASS_NAME, 'activity-option-cart-message-wrapper').text
            option_price_per_person = 'Not Available'
        try:
            spots_left = option.find_element(By.CSS_SELECTOR, "span[data-test-id*='activity-option-is-x-spots-left']").text
        except:
            spots_left = 'N/A'

        list_of_items.append({
            'extraction_date': date_today,
            'date': option_date,
            'title': activity_title,
            'option': option_title,
            'time_range': option_time_range,
            'total_price': option_price_total,
            'price_per_person': option_price_per_person,
            'language': language,
            'adults': adults_amount,
            'spots_left': spots_left,
            'title_url': url
            })    
        logger_done.info(f'Sucessfuly extracted | {option_title} | {option_time_range} | {option_price_total} | {option_price_per_person} |')
    return list_of_items


In [ ]:
def check_if_current_day_done(url_unique_identifier):
    # Constructing the file path
    file_path = f'{output_gyg}/{date_today}-{url_unique_identifier}-GYG.csv'
    
    # Check if the file exists
    if os.path.exists(file_path):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Assuming 'date_processed' is the name of the column containing the dates
        if 'date' in df.columns:
            # Convert the 'date_processed' column to datetime format
            df['date'] = pd.to_datetime(df['date'])
            
            # Find the maximum date
            max_date = df['date'].max()
            
            # Returning True to indicate the task is already processed, and the max date
            return True, max_date.date()  # Converting datetime to date for easier comparison
    # If the file does not exist or 'date_processed' column is not found, return False and None
    return False, None

In [ ]:
# def get_future_price(language, adults_amount,  max_days_to_complete):
df_links = pd.read_csv(link_file)
EUR_City = [
    "Amsterdam", "Athens", "Barcelona", "Berlin", "Dublin", "Dubrovnik", "Florence", "Istanbul",
    "Krakow", "Lisbon", "Madrid", "Milan", "Naples", "Paris", "Porto", "Rome", "Palermo", "Venice",
    "Taormina", "Capri", "Sorrento", "Mount-Etna", "Mount-Vesuvius", "Herculaneum", "Amalfi-Coast",
    "Pompeii"
]

USD_City = [
    "Las-Vegas", "New-York-City", "Cancun", "Dubai"
]

GBP_City = [
    "Edinburgh", "London"
]
df_links = df_links[df_links['Run'] == 1]
driver = initilize_driver()

for index, row in df_links.iterrows():
    url = row['URL']
    url_unique_identifier = url.split('.com/')[-1].split('/')[0]
    language = 'en'
    adults_amount = '2'
    url_detials = f'?lang={language}&date_from={date_today}&_pc=1,{adults_amount}'
    url = url + url_detials

    # Set the number of days to complete the task
    max_days_to_complete = 2

    # Calculate the future date by adding max_days_to_complete to the current date, then extract the day number
    # and convert it to a string. This represents the day number after the specified days are added to the current date.
    picked_max_day_number = str((datetime.datetime.now() + datetime.timedelta(days=max_days_to_complete)).day)

    # Calculate the future date by adding max_days_to_complete to the current date
    # This is used to get a complete date object for further manipulation or formatting.
    picked_max_date_obj = (datetime.datetime.now() + datetime.timedelta(days=max_days_to_complete))

    # Format the future date object as a string in the 'YYYY-MM-DD' format
    # This provides a standard date format that can be used for display or storage.
    picked_max_date = picked_max_date_obj.strftime('%Y-%m-%d')

    # Calculate the number of months to complete the task by subtracting the current month from the month
    # of the future date and adding 1. This gives the total number of months covering the period from the current date to the future date.
    # Note: This calculation assumes the task spans within a single year and is intended for short-term calculations.
    month_to_complete = picked_max_date_obj.month - datetime.datetime.now().month + 1

    driver.get(url)
    print(url)
    is_done, max_date_done = check_if_current_day_done(url_unique_identifier)
    if is_done:
        logger_done.info(f'Url was already done today with max date: {max_date_done}')
        continue
    #     VERIFY IF THE CURRENCY IS CORRECT
    login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Profile']")))
    login_button.click()

    currency_switcher_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='option option-currency']")))

    change_currency(currency_switcher_button, 'EUR')
    #css selector for the box where is availability
    activity_title = driver.find_element(By.CSS_SELECTOR, "h1[data-track='activity-title']").text
    css_selector_booking_tile = f"div[data-track='booking-assistant']"
    #Initiate booking tile varaible 
    booking_tile = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_booking_tile)))
    #Initiate calendar picker 
    calendar_picker = booking_tile.find_element(By.CSS_SELECTOR,"section[class='ba-dropdown ba-date-picker']")

    # css selector for the button check availability
    css_selector_check_availability = f"button[class*='js-check-availability']"
    button_check_availability = WebDriverWait(booking_tile, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_check_availability)))

    #Click  Check availability (once only)
    driver.execute_script("arguments[0].click();", button_check_availability)

    try:
        WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CLASS_NAME, 'dayContainer')))
        months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
        for month in months:
            days_available = month.find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day-available]")
            for day in days_available:
                # Scroll the day into view using JavaScript
                # driver.execute_script("arguments[0].scrollIntoView();", day)
                # Alternatively, you can use ActionChains to move to the element before clicking
                ActionChains(driver).move_to_element(day).perform()
                day.click()
                driver.execute_script("arguments[0].click();", button_check_availability)
                break
            break
    except:
        print('Current date is avialable')
        pass
        # After click on the element wait for the option detials to show up
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'details')))
    option_detials = driver.find_elements(By.TAG_NAME, 'details')
    option_date = driver.current_url.split('date_from=')[-1].split('&')[0]
    current_date = option_date.split('-')[-1]
    print(current_date)
    list_of_items = extract_options(option_detials)
    ### After extraction transform to dataframe and save it in the CSV file in case of any error in th future
    df = pd.DataFrame(list_of_items)
    display(df)
    ## SAVE
    df = save_and_erase_dataframe(df, url_unique_identifier)
    calendar_picker.click()
    #Wait until the calendar is fully loaded
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'dayContainer')))
    months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
    month_done = False
    
    month = months[0]
    for i in range(0, month_to_complete):
        try:
            month_last_day = month.text.split()[-1]
        except:
            months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
            month = months[1]
            month_last_day = month.text.split()[-1]

        days_available = month.find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day-available]") 
        for day in days_available:
            try:
                day_text = day.text.strip()
            except:
                months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
                month = months[0]
                day = month.find_element(By.CSS_SELECTOR, f"span[aria-label*='{next_day}']") 
                day_text = day.text.strip()

                    
            
            print(f"Len: {len(day_text)} - {day_text}")
            if len(day_text) > 0:
                day_date = day.get_attribute('aria-label')
                next_day = (datetime.datetime.strptime(day_date, '%B %d, %Y') + datetime.timedelta(days=1)).strftime('%B %d, %Y').replace(' 0', ' ')
                day.click()
                WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'details')))
                option_detials = driver.find_elements(By.TAG_NAME, 'details')
                time.sleep(2)
                list_of_items = extract_options(option_detials)
                ### After extraction transform to dataframe and save it in the CSV file in case of any error in th future
                df = pd.DataFrame(list_of_items)
                display(df)
                ## SAVE
                df = save_and_erase_dataframe(df, url_unique_identifier)
            try:
                modal_close_button = driver.find_element(By.CSS_SELECTOR,"button[class='tfe-modal-header__close']")
                modal_close_button.click()
                time.sleep(2)
                driver.execute_script("arguments[0].scrollIntoView(true);", calendar_picker)
                calendar_picker.click()
            except:
                driver.execute_script("arguments[0].scrollIntoView(true);", calendar_picker)
                calendar_picker.click()

            print(day_text, '----' , picked_max_day_number, day_text == picked_max_day_number)
            if day_text == picked_max_day_number or day_text == month_last_day:
                i+=1
                break   

quit_driver(driver)

In [ ]:
# combine processed files:
def process_csv_files(folder_path):
    # Determine today's date in the desired format    

    output_file_path = os.path.join(folder_path, f"{date_today}_future_price.xlsx")
    if os.path.exists(output_file_path):
        print("Output file already exists. Exiting function.")
        return
    # Ensure the archive directory exists
    archive_path = os.path.join(folder_path, "archive")
    os.makedirs(archive_path, exist_ok=True)
    
    # Initialize an empty DataFrame to hold all the data
    combined_df = pd.DataFrame()
    
    # Iterate over files in the folder
    for filename in os.listdir(folder_path):
        if filename.startswith(date_today) and filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            
            # Read the CSV file and append its contents to the DataFrame
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            
            # Move the processed file to the archive folder
            shutil.move(file_path, os.path.join(archive_path, filename))
    
    # Save the combined DataFrame to a new CSV file
    combined_df.to_excel(output_file_path, index=False)

    print(f"All data has been combined and saved to {output_file_path}.")


process_csv_files(output_gyg)


In [86]:
# get_future_price(language='en', adults_amount='2', max_days_to_complete=2)
def upload_excel_to_azure_storage_account(local_file_path, storage_account_name, storage_account_key, container_name_raw, blob_name):
    try:
        # Create a connection string to the Azure Storage account
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"

        # Create a BlobServiceClient object using the connection string
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)

        # Get a reference to the container
        container_client = blob_service_client.get_container_client(container_name_raw)

        # Upload the file to Azure Blob Storage
        with open(local_file_path, "rb") as file:
            container_client.upload_blob(name=blob_name, data=file)
        create_log_done('Raw')
        print("File uploaded successfully to Azure Blob Storage (raw).")

    except Exception as e:
        print(f"An error occurred: {e}")

In [87]:
def extract_date_from_price(text):
    if 'Next available date:' in text:
        # Extract the date using string manipulation
        date_part = text.split('Next available date: ')[1].strip()
        
        # Parse the date string into a datetime object
        try:
            date_obj = datetime.datetime.strptime(date_part, "%A, %B %d, %Y")
            # Format the datetime object to "YYYY-MM-DD"
            return date_obj.strftime("%Y-%m-%d")
        except ValueError:
            print("Date format mismatch or error in parsing date.")
            return np.nan
    else:
        return np.nan
    
# Dynamic function to check the 'option' and set message
def dynamic_message_option(option):
    # Define patterns and their corresponding messages
    messages = {
        'Please select 1 participants or fewer for this activity.': 'set adults to 1',
        'Maximum 2 adults allowed per booking': 'adults set to 2'
        # Add more patterns and messages as needed
    }
    
    # Check each pattern and return the corresponding message if found
    for pattern, message in messages.items():
        if pattern in option:
            return message
    
    # Default return if no patterns matched
    return np.nan

def set_to_long_price_to_nan(text):
    if len(text) > 15:
        return np.nan
    else:
        return text

In [91]:
def transform_upload_to_refined(local_file_path, storage_account_name, storage_account_key, container_name_refined, blob_name):
    # Define the Azure Blob Storage connection details
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
    # Read the Excel file into a Pandas DataFrame
    df = pd.read_excel(local_file_path)

    # Make changes to the df DataFrame as needed
    df['extraction_date'] = df['extraction_date'].astype('str')
    df['date'] = df['date'].astype('str')
    df['availability'] = df['total_price'].apply(extract_date_from_price)
    df['message'] = df['total_price'].apply(dynamic_message_option)
    df['total_price'] = df['total_price'].apply(set_to_long_price_to_nan)
    df['price_per_person'] = df['price_per_person'].replace('Not Available', np.nan)
    df['total_price'] = df['total_price'].str.replace(r'[$€£]', '', regex=True).str.replace(',', '').str.strip()
    # df['total_price'] = df['total_price'].map(lambda x: x.split(x[0]) [1].strip() if not x[0].isnumeric() else x)
    # df['price_per_person'] = df['price_per_person'].map(lambda x: x.split(x[0])[1].strip() if not x[0].isnumeric() else x))

    # Save modified DataFrame to an Excel file temporarily
    output_file_path = "temp_modified_excel.xlsx"
    df.to_excel(output_file_path, index=False)
    # Create a connection to Azure Blob Storage
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        container_client = blob_service_client.get_container_client(container_name_refined)
        with open(output_file_path, "rb") as data:
            container_client.upload_blob(name=blob_name, data=data, overwrite=True)
        print("File uploaded successfully to Azure Blob Storage (refined).")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Optional: Remove the temporary file if no longer needed
        os.remove(output_file_path)

    # Function for logging (create_log_done) needs to be defined or replaced
    create_log_done('Refined')

    return 'Added to Blob'


In [92]:
output_file_path = os.path.join(output_gyg, f"{date_today}_future_price.xlsx")
upload_excel_to_azure_storage_account(output_file_path, storage_account_name, storage_account_key, container_name_raw, blob_name)
transform_upload_to_refined(output_file_path, storage_account_name, storage_account_key, container_name_refined, blob_name)

File uploaded successfully to Azure Blob Storage (raw).
File uploaded successfully to Azure Blob Storage (refined).


'Added to Blob'

In [ ]:
# url = df_links.at[0, 'URL']
# url_unique_identifier = url.split('.com/')[-1].split('/')[0]
# language = 'en'
# adults_amount = '2'
# url_detials = f'?lang={language}&date_from={date_today}&_pc=1,{adults_amount}'
# url = url + url_detials
# picked_max_day_number = '1'

# driver.get(url)
# print(url)
# #     VERIFY IF THE CURRENCY IS CORRECT
# login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Profile']")))
# #             Below is previous version when the it was Log in instead Prfile
# #             login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Log in']")))
# login_button.click()
# #             currency = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Select Currency']")))
# login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Profile']")))
# login_button.click()
# #                     currency_switcher_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@title='Select Currency']")))
# currency_switcher_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='option option-currency']")))

# change_currency(currency_switcher_button, 'EUR')
# #css selector for the box where is availability
# activity_title = driver.find_element(By.CSS_SELECTOR, "h1[data-track='activity-title']").text
# css_selector_booking_tile = f"div[data-track='booking-assistant']"
# #Initiate booking tile varaible 
# booking_tile = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_booking_tile)))
# #Initiate calendar picker 
# calendar_picker = booking_tile.find_element(By.CSS_SELECTOR,"section[class='ba-dropdown ba-date-picker']")

# # css selector for the button check availability
# css_selector_check_availability = f"button[class*='js-check-availability']"
# button_check_availability = WebDriverWait(booking_tile, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, css_selector_check_availability)))

# #Click  Check availability (once only)
# driver.execute_script("arguments[0].click();", button_check_availability)

# try:
#     WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CLASS_NAME, 'dayContainer')))
#     months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
#     for month in months:
#         days_available = month.find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day-available]")
#         for day in days_available:
#             # Scroll the day into view using JavaScript
#             # driver.execute_script("arguments[0].scrollIntoView();", day)
#             # Alternatively, you can use ActionChains to move to the element before clicking
#             ActionChains(driver).move_to_element(day).perform()
#             day.click()
#             driver.execute_script("arguments[0].click();", button_check_availability)
#             break
#         break
# except:
#     print('Current date is avialable')
#     pass

    

In [ ]:
# # After click on the element wait for the option detials to show up
# WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'details')))
# option_detials = driver.find_elements(By.TAG_NAME, 'details')
# option_date = driver.current_url.split('date_from=')[-1].split('&')[0]
# current_date = option_date.split('-')[-1]
# print(current_date)
# list_of_items = extract_options(option_detials)
# ### After extraction transform to dataframe and save it in the CSV file in case of any error in th future
# df = pd.DataFrame(list_of_items)
# display(df)
# ## SAVE
# df = save_and_erase_dataframe(df, url_unique_identifier)
# calendar_picker.click()
# #Wait until the calendar is fully loaded
# WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'dayContainer')))
# months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
# month_done = False
# month_to_complete = 2
# month = months[0]
# for i in range(0, month_to_complete):
#     try:
#         month_last_day = month.text.split()[-1]
#     except:
#         months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
#         month = months[1]
#         month_last_day = month.text.split()[-1]


#     # if month_done == True:
#     #     months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
#     #     days_available = months[1].find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day-available]")
#     #     print('Days from new month')
#     # else:
#     days_available = month.find_elements(By.CSS_SELECTOR, "span[data-test-id=ba-calendar-day-available]") 
#     for day in days_available:
#         try:
#             day_text = day.text.strip()
#         except:
#             months = driver.find_elements(By.CLASS_NAME, 'dayContainer')
#             month = months[0]
#             day = month.find_element(By.CSS_SELECTOR, f"span[aria-label*='{next_day}']") 
#             day_text = day.text.strip()

                
        
#         print(f"Len: {len(day_text)} - {day_text}")
#         if len(day_text) > 0:
#             day_date = day.get_attribute('aria-label')
#             next_day = (datetime.datetime.strptime(day_date, '%B %d, %Y') + datetime.timedelta(days=1)).strftime('%B %d, %Y').replace(' 0', ' ')
#             day.click()
#             WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'details')))
#             option_detials = driver.find_elements(By.TAG_NAME, 'details')
#             time.sleep(2)
#             list_of_items = extract_options(option_detials)
#             ### After extraction transform to dataframe and save it in the CSV file in case of any error in th future
#             df = pd.DataFrame(list_of_items)
#             display(df)
#             ## SAVE
#             df = save_and_erase_dataframe(df, url_unique_identifier)
#         try:
#             modal_close_button = driver.find_element(By.CSS_SELECTOR,"button[class='tfe-modal-header__close']")
#             modal_close_button.click()
#             time.sleep(2)
#             calendar_picker.click()
#         except:
#             calendar_picker.click()

#         print(day_text, '----' , picked_max_day_number, day_text == picked_max_day_number)
#         if day_text == picked_max_day_number or day_text == month_last_day:
#             i+=1
#             break
    
